In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Import Library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 데이터셋 합치기

In [ ]:
# Read Data
cj = pd.read_excel('./gdrive/My Drive/빅콘 대상팀/data/CJ/CJ.xlsx', error_bad_lines=False)
gs = pd.read_csv('./gdrive/My Drive/빅콘 대상팀/data/GS/GS.csv',  encoding='cp949')
scr =  pd.read_csv('./gdrive/My Drive/빅콘 대상팀/data/SC/CARD_SPENDING_RESIDENT.txt', sep='\t')
skt =  pd.read_csv('./gdrive/My Drive/빅콘 대상팀/data/SK/skt.csv', index_col='Unnamed: 0')
ska = pd.read_csv('./gdrive/My Drive/빅콘 대상팀/data/SK/ska.csv', index_col='Unnamed: 0')

In [ ]:
# scr
scr.rename(columns = {'STD_DD' : 'STD_YMD'}, inplace = True)
scr.rename(columns = {'DONG_CD' : 'HDONG_CD'}, inplace = True)

scr['STD_YMD'] = pd.to_datetime(scr['STD_YMD'].astype(str), format='%Y%m%d')
scr = scr.drop(['GU_CD', 'SEX_CD', 'AGE_CD'], axis=1) 

def name_a_to_b(data,col,a,b):
  data = data.replace({col:a}, {col:b}, inplace=True)
  return(data)

cat_code = [10,20,21,22,30,31,32,33,34,35,40,42,43,44,50,52,60,62,70,71,80,81,92]
cat_name = ['숙박','레저용품','레저업소','문화취미','가구','전기','주방용구',
            '연료판매','광학제품','가전','유통업','의복','직물','신변잡화','서적문구','사무통신',
            '자동차판매','자동차정비','의료기관','보건위생','요식업소','음료식품','수리서비스']
            
for i in range(len(cat_code)):
  name_a_to_b(scr,'MCT_CAT_CD',cat_code[i],cat_name[i])

scr = scr.groupby(['STD_YMD', 'HDONG_CD', 'MCT_CAT_CD']).sum().unstack(level=2).fillna(0)
scr.head()

USE_CNT                      ... USE_AMT                         
MCT_CAT_CD               가구     가전  광학제품   레저업소  ...   자동차판매      전기     주방용구      직물
STD_YMD    HDONG_CD                              ...                                 
2019-02-01 510          0.0   31.0   0.0  166.0  ...     0.0     0.0      0.0     0.0
           517          0.0    0.0   5.0   97.0  ...     0.0  1320.0    275.0     0.0
           520          0.0    0.0  29.0  400.0  ...     0.0     0.0    276.0     0.0
           530          0.0    0.0   0.0  204.0  ...     0.0     0.0      0.0     0.0
           540          0.0  241.0  86.0  319.0  ...     0.0     0.0  19667.0  1278.0

[5 rows x 46 columns]

In [ ]:
# gs
gs.rename(columns = {'OPER_DT' : 'STD_YMD'}, inplace = True)
gs.rename(columns = {'ADMD' : 'HDONG_NM'}, inplace = True)

gs['STD_YMD'] = pd.to_datetime(gs['STD_YMD'].astype(str), format='%Y%m%d')

gs = gs.drop(['PVN', 'BOR'], axis=1)
gs.head()

,STD_YMD,HDONG_NM,AMT_IND,LCLS_10_P,LCLS_20_P,LCLS_30_P,LCLS_40_P,LCLS_50_P,LCLS_60_P,LCLS_70_P,LCLS_80_P
0,2019-02-01,고산1동,1.06,45.10%,17.20%,29.50%,2.30%,4.50%,0.50%,0.50%,0.40%
1,2019-02-01,고산2동,1.08,45.80%,19.60%,29.60%,0.60%,3.40%,0.10%,0.90%,0.10%
2,2019-02-01,고산3동,0.86,36.00%,19.70%,37.70%,1.30%,3.00%,0.90%,0.90%,0.60%
3,2019-02-01,두산동,0.72,28.20%,17.50%,36.30%,2.40%,13.40%,0.00%,2.10%,0.00%
4,2019-02-01,만촌1동,1.16,40.10%,20.00%,31.10%,3.90%,3.50%,0.50%,0.60%,0.30%


In [ ]:
cj.rename(columns = {'DL_YMD' : 'STD_YMD'}, inplace = True)
cj.rename(columns = {'DL_GD_LCLS_NM' : 'CJ_CATG'}, inplace = True)
cj.rename(columns = {'HDNG_NM' : 'HDONG_NM'}, inplace = True)
cj.rename(columns = {'HDNG_CD' : 'HDONG_CD'}, inplace = True)

cj['STD_YMD'] = pd.to_datetime(cj['STD_YMD'].astype(str), format='%y%m%d')

cj['HDONG_NM'] = cj['HDONG_NM'].apply(lambda x: '신당5동' if x == '신당제5동' else x)

cj = cj.drop(['DL_GD_LCLS_CD', 'CTPV_NM', 'CTPV_CD', 'CTGG_NM', 'CTGG_CD'], axis=1)
cj = cj.set_index(['STD_YMD', 'HDONG_CD', 'HDONG_NM', 'CJ_CATG']).unstack(level=3).fillna(0)
cj = cj.reset_index(drop=False).set_index(['STD_YMD', 'HDONG_CD'])
cj.head()

HDONG_NM INVC_CONT               ...                         
CJ_CATG                        가구/인테리어 도서/음반 디지털/가전  ... 출산/육아  패션의류  패션잡화 화장품/미용
STD_YMD    HDONG_CD                                  ...                         
2019-02-01 510          범어1동       6.0  64.0   15.0  ...   0.0  29.0  12.0   19.0
           517           동인동       6.0  41.0    6.0  ...   1.0  21.0   9.0   21.0
           520          범어2동      12.0  53.0   14.0  ...   0.0  36.0  19.0   28.0
           520           소공동       3.0  48.0   35.0  ...   0.0  25.0  10.0   26.0
           530          범어3동       4.0  46.0   10.0  ...   1.0  39.0  11.0   22.0

[5 rows x 11 columns]

In [ ]:
skt['STD_YMD'] = pd.to_datetime(skt['STD_YMD'].astype(str), format='%Y%m%d') # dtype datetime으로 변경

skt['TMST_MORNING'] = np.mean(skt.iloc[:,15:19], axis=1)
skt['TMST_NOON'] = np.mean(skt.iloc[:,15:18], axis=1)
skt['TMST_AFTNOON'] = np.mean(skt.iloc[:,18:22], axis=1)
skt['TMST_NIGHT'] = np.mean(skt.iloc[:,22:26], axis=1)
skt['TMST_DAWN'] = np.mean(pd.concat([skt.iloc[:, 26:28], skt.iloc[:, 5:11]], axis=1), axis=1)

skt = skt[['STD_YMD', 'HDONG_NM', 'TMST_MORNING', 'TMST_NOON', 'TMST_AFTNOON', 'TMST_NIGHT', 'TMST_DAWN']]
skt.head()

# skt = skt.set_index(['HDONG_NM', 'STD_YMD']) # index로 설정

# ska_backup = ska
# ska['STD_YMD'] = pd.to_datetime(ska['STD_YMD'].astype(str), format='%Y%m%d') # dtype datetime으로 변경
# ska = ska.set_index(['HDONG_NM', 'STD_YMD']) # index로 설정

# sk = pd.concat([skt, ska.iloc[:,3:]], axis=1)

,STD_YMD,HDONG_NM,TMST_MORNING,TMST_NOON,TMST_AFTNOON,TMST_NIGHT,TMST_DAWN
0,2019-02-01,소공동,12642.9050,12738.570000,12449.9950,8822.2900,3058.72000
1,2019-02-01,회현동,19121.4125,19425.116667,18121.2175,12317.6600,3884.33125
2,2019-02-01,명동,21962.5875,22505.880000,20520.2625,13770.9950,4681.89875
3,2019-02-01,필동,7209.0200,7311.806667,9043.2975,6588.2700,1675.17125
4,2019-02-01,장충동,4912.9025,4821.530000,5224.2850,3927.0175,1226.98000


In [ ]:
# cj랑 sc 코드로 합치고, 코드 지우기
all = pd.merge(cj, scr, left_on=['STD_YMD', 'HDONG_CD'], right_on=['STD_YMD', 'HDONG_CD'], how='left')
all = all.reset_index(drop=False).set_index(['STD_YMD', 'HDONG_NM']).drop('HDONG_CD', axis=1)
all = pd.merge(all, gs, left_on=['STD_YMD', 'HDONG_NM'], right_on=['STD_YMD', 'HDONG_NM'], how='left')
all = pd.merge(all, skt, left_on=['STD_YMD', 'HDONG_NM'], right_on=['STD_YMD', 'HDONG_NM'], how='left')
all.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
/usr/local/lib/python3.6/dist-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


,STD_YMD,HDONG_NM,"(INVC_CONT, 가구/인테리어)","(INVC_CONT, 도서/음반)","(INVC_CONT, 디지털/가전)","(INVC_CONT, 생활건강)","(INVC_CONT, 스포츠/레저)","(INVC_CONT, 식품)","(INVC_CONT, 출산/육아)","(INVC_CONT, 패션의류)","(INVC_CONT, 패션잡화)","(INVC_CONT, 화장품/미용)","(USE_CNT, 가구)","(USE_CNT, 가전)","(USE_CNT, 광학제품)","(USE_CNT, 레저업소)","(USE_CNT, 레저용품)","(USE_CNT, 문화취미)","(USE_CNT, 보건위생)","(USE_CNT, 사무통신)","(USE_CNT, 서적문구)","(USE_CNT, 수리서비스)","(USE_CNT, 숙박)","(USE_CNT, 신변잡화)","(USE_CNT, 연료판매)","(USE_CNT, 요식업소)","(USE_CNT, 유통업)","(USE_CNT, 음료식품)","(USE_CNT, 의료기관)","(USE_CNT, 의복)","(USE_CNT, 자동차정비)","(USE_CNT, 자동차판매)","(USE_CNT, 전기)","(USE_CNT, 주방용구)","(USE_CNT, 직물)","(USE_AMT, 가구)","(USE_AMT, 가전)","(USE_AMT, 광학제품)","(USE_AMT, 레저업소)","(USE_AMT, 레저용품)","(USE_AMT, 문화취미)","(USE_AMT, 보건위생)","(USE_AMT, 사무통신)","(USE_AMT, 서적문구)","(USE_AMT, 수리서비스)","(USE_AMT, 숙박)","(USE_AMT, 신변잡화)","(USE_AMT, 연료판매)","(USE_AMT, 요식업소)","(USE_AMT, 유통업)","(USE_AMT, 음료식품)","(USE_AMT, 의료기관)","(USE_AMT, 의복)","(USE_AMT, 자동차정비)","(USE_AMT, 자동차판매)","(USE_AMT, 전기)","(USE_AMT, 주방용구)","(USE_AMT, 직물)",AMT_IND,LCLS_10_P,LCLS_20_P,LCLS_30_P,LCLS_40_P,LCLS_50_P,LCLS_60_P,LCLS_70_P,LCLS_80_P,TMST_MORNING,TMST_NOON,TMST_AFTNOON,TMST_NIGHT,TMST_DAWN
0,2019-02-01,범어1동,6.0,64.0,15.0,20.0,3.0,14.0,0.0,29.0,12.0,19.0,0.0,31.0,0.0,166.0,0.0,599.0,237.0,0.0,0.0,26.0,46.0,0.0,908.0,3938.0,3056.0,627.0,1521.0,0.0,197.0,0.0,0.0,0.0,0.0,0.0,12158.0,0.0,25635.0,0.0,43048.0,12552.0,0.0,0.0,942.0,2010.0,0.0,29133.0,117524.0,44322.0,16206.0,132778.0,0.0,2443.0,0.0,0.0,0.0,0.0,1.08,31.50%,16.80%,39.10%,1.60%,9.30%,0.10%,1.20%,0.30%,6613.0625,6470.653333,6570.0875,5895.6525,1109.99625
1,2019-02-01,동인동,6.0,41.0,6.0,20.0,0.0,15.0,1.0,21.0,9.0,21.0,0.0,0.0,5.0,97.0,5.0,0.0,73.0,20.0,0.0,5.0,54.0,0.0,330.0,2640.0,2398.0,379.0,1022.0,0.0,52.0,0.0,5.0,15.0,0.0,0.0,0.0,27.0,3268.0,454.0,0.0,8949.0,804.0,0.0,26.0,2001.0,0.0,13469.0,63156.0,20924.0,30160.0,136270.0,0.0,2429.0,0.0,1320.0,275.0,0.0,0.77,37.50%,16.80%,32.80%,6.20%,6.40%,0.20%,0.20%,0.00%,6020.9925,5671.933333,6034.2100,4751.1775,927.58250
2,2019-02-01,범어2동,12.0,53.0,14.0,23.0,0.0,20.0,0.0,36.0,19.0,28.0,0.0,0.0,29.0,400.0,30.0,537.0,558.0,72007.0,379.0,54.0,960.0,48.0,566.0,184667.0,61985.0,1969.0,3316.0,142.0,6409.0,0.0,0.0,5.0,0.0,0.0,0.0,966.0,20710.0,8766.0,7935.0,31642.0,514292.0,7774.0,1495.0,115004.0,51541.0,46888.0,1781639.0,2300513.0,69906.0,302176.0,14825.0,111690.0,0.0,0.0,276.0,0.0,1.06,42.80%,19.60%,27.00%,2.50%,6.80%,0.00%,1.00%,0.40%,6584.1325,6379.133333,7065.8275,5671.3975,1115.14500
3,2019-02-01,소공동,3.0,48.0,35.0,15.0,0.0,19.0,0.0,25.0,10.0,26.0,0.0,0.0,29.0,400.0,30.0,537.0,558.0,72007.0,379.0,54.0,960.0,48.0,566.0,184667.0,61985.0,1969.0,3316.0,142.0,6409.0,0.0,0.0,5.0,0.0,0.0,0.0,966.0,20710.0,8766.0,7935.0,31642.0,514292.0,7774.0,1495.0,115004.0,51541.0,46888.0,1781639.0,2300513.0,69906.0,302176.0,14825.0,111690.0,0.0,0.0,276.0,0.0,1.67,28.80%,18.60%,38.40%,4.20%,8.10%,0.40%,1.10%,0.50%,12642.9050,12738.570000,12449.9950,8822.2900,3058.72000
4,2019-02-01,범어3동,4.0,46.0,10.0,21.0,1.0,31.0,1.0,39.0,11.0,22.0,0.0,0.0,0.0,204.0,15.0,47.0,36.0,5.0,47.0,96.0,20.0,0.0,0.0,2597.0,2336.0,585.0,1466.0,0.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9299.0,276.0,1173.0,594.0,5.0,1004.0,993.0,975.0,0.0,0.0,87056.0,32592.0,13834.0,100590.0,0.0,877.0,0.0,0.0,0.0,0.0,0.96,30.90%,21.60%,32.70%,1.70%,9.00%,0.70%,0.30%,3.10%,3900.8175,3779.263333,4149.5600,3604.3350,686.79000


In [ ]:
#all.to_csv('./gdrive/My Drive/빅콘 대상팀/data/all.csv', sep=',',index=False)

In [ ]:
all = pd.read_csv('./gdrive/My Drive/빅콘 대상팀/data/all.csv')

In [ ]:
all.head()

,STD_YMD,HDONG_NM,"('INVC_CONT', '가구/인테리어')","('INVC_CONT', '도서/음반')","('INVC_CONT', '디지털/가전')","('INVC_CONT', '생활건강')","('INVC_CONT', '스포츠/레저')","('INVC_CONT', '식품')","('INVC_CONT', '출산/육아')","('INVC_CONT', '패션의류')","('INVC_CONT', '패션잡화')","('INVC_CONT', '화장품/미용')","('USE_CNT', '가구')","('USE_CNT', '가전')","('USE_CNT', '광학제품')","('USE_CNT', '레저업소')","('USE_CNT', '레저용품')","('USE_CNT', '문화취미')","('USE_CNT', '보건위생')","('USE_CNT', '사무통신')","('USE_CNT', '서적문구')","('USE_CNT', '수리서비스')","('USE_CNT', '숙박')","('USE_CNT', '신변잡화')","('USE_CNT', '연료판매')","('USE_CNT', '요식업소')","('USE_CNT', '유통업')","('USE_CNT', '음료식품')","('USE_CNT', '의료기관')","('USE_CNT', '의복')","('USE_CNT', '자동차정비')","('USE_CNT', '자동차판매')","('USE_CNT', '전기')","('USE_CNT', '주방용구')","('USE_CNT', '직물')","('USE_AMT', '가구')","('USE_AMT', '가전')","('USE_AMT', '광학제품')","('USE_AMT', '레저업소')","('USE_AMT', '레저용품')","('USE_AMT', '문화취미')","('USE_AMT', '보건위생')","('USE_AMT', '사무통신')","('USE_AMT', '서적문구')","('USE_AMT', '수리서비스')","('USE_AMT', '숙박')","('USE_AMT', '신변잡화')","('USE_AMT', '연료판매')","('USE_AMT', '요식업소')","('USE_AMT', '유통업')","('USE_AMT', '음료식품')","('USE_AMT', '의료기관')","('USE_AMT', '의복')","('USE_AMT', '자동차정비')","('USE_AMT', '자동차판매')","('USE_AMT', '전기')","('USE_AMT', '주방용구')","('USE_AMT', '직물')",AMT_IND,LCLS_10_P,LCLS_20_P,LCLS_30_P,LCLS_40_P,LCLS_50_P,LCLS_60_P,LCLS_70_P,LCLS_80_P,TMST_MORNING,TMST_NOON,TMST_AFTNOON,TMST_NIGHT,TMST_DAWN
0,2019-02-01,범어1동,6.0,64.0,15.0,20.0,3.0,14.0,0.0,29.0,12.0,19.0,0.0,31.0,0.0,166.0,0.0,599.0,237.0,0.0,0.0,26.0,46.0,0.0,908.0,3938.0,3056.0,627.0,1521.0,0.0,197.0,0.0,0.0,0.0,0.0,0.0,12158.0,0.0,25635.0,0.0,43048.0,12552.0,0.0,0.0,942.0,2010.0,0.0,29133.0,117524.0,44322.0,16206.0,132778.0,0.0,2443.0,0.0,0.0,0.0,0.0,1.08,31.50%,16.80%,39.10%,1.60%,9.30%,0.10%,1.20%,0.30%,6613.0625,6470.653333,6570.0875,5895.6525,1109.99625
1,2019-02-01,동인동,6.0,41.0,6.0,20.0,0.0,15.0,1.0,21.0,9.0,21.0,0.0,0.0,5.0,97.0,5.0,0.0,73.0,20.0,0.0,5.0,54.0,0.0,330.0,2640.0,2398.0,379.0,1022.0,0.0,52.0,0.0,5.0,15.0,0.0,0.0,0.0,27.0,3268.0,454.0,0.0,8949.0,804.0,0.0,26.0,2001.0,0.0,13469.0,63156.0,20924.0,30160.0,136270.0,0.0,2429.0,0.0,1320.0,275.0,0.0,0.77,37.50%,16.80%,32.80%,6.20%,6.40%,0.20%,0.20%,0.00%,6020.9925,5671.933333,6034.2100,4751.1775,927.58250
2,2019-02-01,범어2동,12.0,53.0,14.0,23.0,0.0,20.0,0.0,36.0,19.0,28.0,0.0,0.0,29.0,400.0,30.0,537.0,558.0,72007.0,379.0,54.0,960.0,48.0,566.0,184667.0,61985.0,1969.0,3316.0,142.0,6409.0,0.0,0.0,5.0,0.0,0.0,0.0,966.0,20710.0,8766.0,7935.0,31642.0,514292.0,7774.0,1495.0,115004.0,51541.0,46888.0,1781639.0,2300513.0,69906.0,302176.0,14825.0,111690.0,0.0,0.0,276.0,0.0,1.06,42.80%,19.60%,27.00%,2.50%,6.80%,0.00%,1.00%,0.40%,6584.1325,6379.133333,7065.8275,5671.3975,1115.14500
3,2019-02-01,소공동,3.0,48.0,35.0,15.0,0.0,19.0,0.0,25.0,10.0,26.0,0.0,0.0,29.0,400.0,30.0,537.0,558.0,72007.0,379.0,54.0,960.0,48.0,566.0,184667.0,61985.0,1969.0,3316.0,142.0,6409.0,0.0,0.0,5.0,0.0,0.0,0.0,966.0,20710.0,8766.0,7935.0,31642.0,514292.0,7774.0,1495.0,115004.0,51541.0,46888.0,1781639.0,2300513.0,69906.0,302176.0,14825.0,111690.0,0.0,0.0,276.0,0.0,1.67,28.80%,18.60%,38.40%,4.20%,8.10%,0.40%,1.10%,0.50%,12642.9050,12738.570000,12449.9950,8822.2900,3058.72000
4,2019-02-01,범어3동,4.0,46.0,10.0,21.0,1.0,31.0,1.0,39.0,11.0,22.0,0.0,0.0,0.0,204.0,15.0,47.0,36.0,5.0,47.0,96.0,20.0,0.0,0.0,2597.0,2336.0,585.0,1466.0,0.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9299.0,276.0,1173.0,594.0,5.0,1004.0,993.0,975.0,0.0,0.0,87056.0,32592.0,13834.0,100590.0,0.0,877.0,0.0,0.0,0.0,0.0,0.96,30.90%,21.60%,32.70%,1.70%,9.00%,0.70%,0.30%,3.10%,3900.8175,3779.263333,4149.5600,3604.3350,686.79000


In [ ]:
gu = pd.read_excel('./gdrive/My Drive/빅콘 대상팀/data/지역데이터/구_동.xlsx')

In [ ]:
all.columns =[ 'STD_YMD', 'HDONG_NM', 'CJ_가구/인테리어',
       'CJ_도서/음반', 'CJ_디지털/가전',
       'CJ_생활건강', 'CJ_스포츠/레저',
       'CJ_식품', 'CJ_출산/육아',
       'CJ_패션의류', 'CJ_패션잡화',
       'CJ_화장품/미용', 'SC_CNT_가구', 'SC_CNT_가전',
       'SC_CNT_광학제품', 'SC_CNT_레저업소', 'SC_CNT_레저용품',
       'SC_CNT_문화취미', 'SC_CNT_보건위생', 'SC_CNT_사무통신',
       'SC_CNT_서적문구', 'SC_CNT_수리서비스', 'SC_CNT_숙박',
       'SC_CNT_신변잡화', 'SC_CNT_연료판매', 'SC_CNT_요식업소',
       'SC_CNT_유통업', 'SC_CNT_음료식품', 'SC_CNT_의료기관',
       'SC_CNT_의복', 'SC_CNT_자동차정비', 'SC_CNT_자동차판매',
       'SC_CNT_전기', 'SC_CNT_주방용구', 'SC_CNT_직물',
       'SC_AMT_가구', 'SC_AMT_가전', 'SC_AMT_광학제품',
       'SC_AMT_레저업소', 'SC_AMT_레저용품', 'SC_AMT_문화취미',
       'SC_AMT_보건위생', 'SC_AMT_사무통신', 'SC_AMT_서적문구',
       'SC_AMT_수리서비스', 'SC_AMT_숙박', 'SC_AMT_신변잡화',
       'SC_AMT_연료판매', 'SC_AMT_요식업소', 'SC_AMT_유통업',
       'SC_AMT_음료식품', 'SC_AMT_의료기관', 'SC_AMT_의복',
       'SC_AMT_자동차정비', 'SC_AMT_자동차판매', 'SC_AMT_전기',
       'SC_AMT_주방용구', 'SC_AMT_직물', 'GS_매출지수', 'GS_식사',
       'GS_간식', 'GS_마실거리', 'GS_홈/리빙', 'GS_헬스/뷰티', 'GS_취미/여가활동',
       'GS_사회활동', 'GS_임신육아', 'TMST_MORNING', 'TMST_NOON', 'TMST_AFTNOON',
       'TMST_NIGHT', 'TMST_DAWN']

In [ ]:
data_final = pd.merge(all,gu,how='left',on = 'HDONG_NM')
data_final.head()

,STD_YMD,HDONG_NM,CJ_가구/인테리어,CJ_도서/음반,CJ_디지털/가전,CJ_생활건강,CJ_스포츠/레저,CJ_식품,CJ_출산/육아,CJ_패션의류,CJ_패션잡화,CJ_화장품/미용,SC_CNT_가구,SC_CNT_가전,SC_CNT_광학제품,SC_CNT_레저업소,SC_CNT_레저용품,SC_CNT_문화취미,SC_CNT_보건위생,SC_CNT_사무통신,SC_CNT_서적문구,SC_CNT_수리서비스,SC_CNT_숙박,SC_CNT_신변잡화,SC_CNT_연료판매,SC_CNT_요식업소,SC_CNT_유통업,SC_CNT_음료식품,SC_CNT_의료기관,SC_CNT_의복,SC_CNT_자동차정비,SC_CNT_자동차판매,SC_CNT_전기,SC_CNT_주방용구,SC_CNT_직물,SC_AMT_가구,SC_AMT_가전,SC_AMT_광학제품,SC_AMT_레저업소,SC_AMT_레저용품,SC_AMT_문화취미,SC_AMT_보건위생,SC_AMT_사무통신,SC_AMT_서적문구,SC_AMT_수리서비스,SC_AMT_숙박,SC_AMT_신변잡화,SC_AMT_연료판매,SC_AMT_요식업소,SC_AMT_유통업,SC_AMT_음료식품,SC_AMT_의료기관,SC_AMT_의복,SC_AMT_자동차정비,SC_AMT_자동차판매,SC_AMT_전기,SC_AMT_주방용구,SC_AMT_직물,GS_매출지수,GS_식사,GS_간식,GS_마실거리,GS_홈/리빙,GS_헬스/뷰티,GS_취미/여가활동,GS_사회활동,GS_임신육아,TMST_MORNING,TMST_NOON,TMST_AFTNOON,TMST_NIGHT,TMST_DAWN,HDONG_GU
0,2019-02-01,범어1동,6.0,64.0,15.0,20.0,3.0,14.0,0.0,29.0,12.0,19.0,0.0,31.0,0.0,166.0,0.0,599.0,237.0,0.0,0.0,26.0,46.0,0.0,908.0,3938.0,3056.0,627.0,1521.0,0.0,197.0,0.0,0.0,0.0,0.0,0.0,12158.0,0.0,25635.0,0.0,43048.0,12552.0,0.0,0.0,942.0,2010.0,0.0,29133.0,117524.0,44322.0,16206.0,132778.0,0.0,2443.0,0.0,0.0,0.0,0.0,1.08,31.50%,16.80%,39.10%,1.60%,9.30%,0.10%,1.20%,0.30%,6613.0625,6470.653333,6570.0875,5895.6525,1109.99625,대구 수성구
1,2019-02-01,동인동,6.0,41.0,6.0,20.0,0.0,15.0,1.0,21.0,9.0,21.0,0.0,0.0,5.0,97.0,5.0,0.0,73.0,20.0,0.0,5.0,54.0,0.0,330.0,2640.0,2398.0,379.0,1022.0,0.0,52.0,0.0,5.0,15.0,0.0,0.0,0.0,27.0,3268.0,454.0,0.0,8949.0,804.0,0.0,26.0,2001.0,0.0,13469.0,63156.0,20924.0,30160.0,136270.0,0.0,2429.0,0.0,1320.0,275.0,0.0,0.77,37.50%,16.80%,32.80%,6.20%,6.40%,0.20%,0.20%,0.00%,6020.9925,5671.933333,6034.2100,4751.1775,927.58250,대구 중구
2,2019-02-01,범어2동,12.0,53.0,14.0,23.0,0.0,20.0,0.0,36.0,19.0,28.0,0.0,0.0,29.0,400.0,30.0,537.0,558.0,72007.0,379.0,54.0,960.0,48.0,566.0,184667.0,61985.0,1969.0,3316.0,142.0,6409.0,0.0,0.0,5.0,0.0,0.0,0.0,966.0,20710.0,8766.0,7935.0,31642.0,514292.0,7774.0,1495.0,115004.0,51541.0,46888.0,1781639.0,2300513.0,69906.0,302176.0,14825.0,111690.0,0.0,0.0,276.0,0.0,1.06,42.80%,19.60%,27.00%,2.50%,6.80%,0.00%,1.00%,0.40%,6584.1325,6379.133333,7065.8275,5671.3975,1115.14500,대구 수성구
3,2019-02-01,소공동,3.0,48.0,35.0,15.0,0.0,19.0,0.0,25.0,10.0,26.0,0.0,0.0,29.0,400.0,30.0,537.0,558.0,72007.0,379.0,54.0,960.0,48.0,566.0,184667.0,61985.0,1969.0,3316.0,142.0,6409.0,0.0,0.0,5.0,0.0,0.0,0.0,966.0,20710.0,8766.0,7935.0,31642.0,514292.0,7774.0,1495.0,115004.0,51541.0,46888.0,1781639.0,2300513.0,69906.0,302176.0,14825.0,111690.0,0.0,0.0,276.0,0.0,1.67,28.80%,18.60%,38.40%,4.20%,8.10%,0.40%,1.10%,0.50%,12642.9050,12738.570000,12449.9950,8822.2900,3058.72000,서울 중구
4,2019-02-01,범어3동,4.0,46.0,10.0,21.0,1.0,31.0,1.0,39.0,11.0,22.0,0.0,0.0,0.0,204.0,15.0,47.0,36.0,5.0,47.0,96.0,20.0,0.0,0.0,2597.0,2336.0,585.0,1466.0,0.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9299.0,276.0,1173.0,594.0,5.0,1004.0,993.0,975.0,0.0,0.0,87056.0,32592.0,13834.0,100590.0,0.0,877.0,0.0,0.0,0.0,0.0,0.96,30.90%,21.60%,32.70%,1.70%,9.00%,0.70%,0.30%,3.10%,3900.8175,3779.263333,4149.5600,3604.3350,686.79000,대구 수성구


In [ ]:
#data_final.to_csv('./gdrive/My Drive/빅콘 대상팀/data/all_clean.csv', sep=',',index=False)

In [ ]:
all = pd.read_csv('./gdrive/My Drive/빅콘 대상팀/data/all_clean.csv' ,parse_dates=['STD_YMD'])

In [ ]:
all.head()

,STD_YMD,HDONG_NM,CJ_가구/인테리어,CJ_도서/음반,CJ_디지털/가전,CJ_생활건강,CJ_스포츠/레저,CJ_식품,CJ_출산/육아,CJ_패션의류,CJ_패션잡화,CJ_화장품/미용,SC_CNT_가구,SC_CNT_가전,SC_CNT_광학제품,SC_CNT_레저업소,SC_CNT_레저용품,SC_CNT_문화취미,SC_CNT_보건위생,SC_CNT_사무통신,SC_CNT_서적문구,SC_CNT_수리서비스,SC_CNT_숙박,SC_CNT_신변잡화,SC_CNT_연료판매,SC_CNT_요식업소,SC_CNT_유통업,SC_CNT_음료식품,SC_CNT_의료기관,SC_CNT_의복,SC_CNT_자동차정비,SC_CNT_자동차판매,SC_CNT_전기,SC_CNT_주방용구,SC_CNT_직물,SC_AMT_가구,SC_AMT_가전,SC_AMT_광학제품,SC_AMT_레저업소,SC_AMT_레저용품,SC_AMT_문화취미,SC_AMT_보건위생,SC_AMT_사무통신,SC_AMT_서적문구,SC_AMT_수리서비스,SC_AMT_숙박,SC_AMT_신변잡화,SC_AMT_연료판매,SC_AMT_요식업소,SC_AMT_유통업,SC_AMT_음료식품,SC_AMT_의료기관,SC_AMT_의복,SC_AMT_자동차정비,SC_AMT_자동차판매,SC_AMT_전기,SC_AMT_주방용구,SC_AMT_직물,GS_매출지수,GS_식사,GS_간식,GS_마실거리,GS_홈/리빙,GS_헬스/뷰티,GS_취미/여가활동,GS_사회활동,GS_임신육아,TMST_MORNING,TMST_NOON,TMST_AFTNOON,TMST_NIGHT,TMST_DAWN,HDONG_GU
0,2019-02-01,범어1동,6.0,64.0,15.0,20.0,3.0,14.0,0.0,29.0,12.0,19.0,0.0,31.0,0.0,166.0,0.0,599.0,237.0,0.0,0.0,26.0,46.0,0.0,908.0,3938.0,3056.0,627.0,1521.0,0.0,197.0,0.0,0.0,0.0,0.0,0.0,12158.0,0.0,25635.0,0.0,43048.0,12552.0,0.0,0.0,942.0,2010.0,0.0,29133.0,117524.0,44322.0,16206.0,132778.0,0.0,2443.0,0.0,0.0,0.0,0.0,1.08,31.50%,16.80%,39.10%,1.60%,9.30%,0.10%,1.20%,0.30%,6613.0625,6470.653333,6570.0875,5895.6525,1109.99625,대구 수성구
1,2019-02-01,동인동,6.0,41.0,6.0,20.0,0.0,15.0,1.0,21.0,9.0,21.0,0.0,0.0,5.0,97.0,5.0,0.0,73.0,20.0,0.0,5.0,54.0,0.0,330.0,2640.0,2398.0,379.0,1022.0,0.0,52.0,0.0,5.0,15.0,0.0,0.0,0.0,27.0,3268.0,454.0,0.0,8949.0,804.0,0.0,26.0,2001.0,0.0,13469.0,63156.0,20924.0,30160.0,136270.0,0.0,2429.0,0.0,1320.0,275.0,0.0,0.77,37.50%,16.80%,32.80%,6.20%,6.40%,0.20%,0.20%,0.00%,6020.9925,5671.933333,6034.2100,4751.1775,927.58250,대구 중구
2,2019-02-01,범어2동,12.0,53.0,14.0,23.0,0.0,20.0,0.0,36.0,19.0,28.0,0.0,0.0,29.0,400.0,30.0,537.0,558.0,72007.0,379.0,54.0,960.0,48.0,566.0,184667.0,61985.0,1969.0,3316.0,142.0,6409.0,0.0,0.0,5.0,0.0,0.0,0.0,966.0,20710.0,8766.0,7935.0,31642.0,514292.0,7774.0,1495.0,115004.0,51541.0,46888.0,1781639.0,2300513.0,69906.0,302176.0,14825.0,111690.0,0.0,0.0,276.0,0.0,1.06,42.80%,19.60%,27.00%,2.50%,6.80%,0.00%,1.00%,0.40%,6584.1325,6379.133333,7065.8275,5671.3975,1115.14500,대구 수성구
3,2019-02-01,소공동,3.0,48.0,35.0,15.0,0.0,19.0,0.0,25.0,10.0,26.0,0.0,0.0,29.0,400.0,30.0,537.0,558.0,72007.0,379.0,54.0,960.0,48.0,566.0,184667.0,61985.0,1969.0,3316.0,142.0,6409.0,0.0,0.0,5.0,0.0,0.0,0.0,966.0,20710.0,8766.0,7935.0,31642.0,514292.0,7774.0,1495.0,115004.0,51541.0,46888.0,1781639.0,2300513.0,69906.0,302176.0,14825.0,111690.0,0.0,0.0,276.0,0.0,1.67,28.80%,18.60%,38.40%,4.20%,8.10%,0.40%,1.10%,0.50%,12642.9050,12738.570000,12449.9950,8822.2900,3058.72000,서울 중구
4,2019-02-01,범어3동,4.0,46.0,10.0,21.0,1.0,31.0,1.0,39.0,11.0,22.0,0.0,0.0,0.0,204.0,15.0,47.0,36.0,5.0,47.0,96.0,20.0,0.0,0.0,2597.0,2336.0,585.0,1466.0,0.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9299.0,276.0,1173.0,594.0,5.0,1004.0,993.0,975.0,0.0,0.0,87056.0,32592.0,13834.0,100590.0,0.0,877.0,0.0,0.0,0.0,0.0,0.96,30.90%,21.60%,32.70%,1.70%,9.00%,0.70%,0.30%,3.10%,3900.8175,3779.263333,4149.5600,3604.3350,686.79000,대구 수성구


In [ ]:
for i in np.arange(-14,-6,1):
  all.iloc[:,i] = all.iloc[:,i].str.replace('%','').astype(float)

In [ ]:
all[all.filter(like='GS').columns[1:]] = all[all.filter(like='GS').columns[1:]].multiply(all["GS_매출지수"], axis="index")

In [ ]:
all.head()

,STD_YMD,HDONG_NM,CJ_가구/인테리어,CJ_도서/음반,CJ_디지털/가전,CJ_생활건강,CJ_스포츠/레저,CJ_식품,CJ_출산/육아,CJ_패션의류,CJ_패션잡화,CJ_화장품/미용,SC_CNT_가구,SC_CNT_가전,SC_CNT_광학제품,SC_CNT_레저업소,SC_CNT_레저용품,SC_CNT_문화취미,SC_CNT_보건위생,SC_CNT_사무통신,SC_CNT_서적문구,SC_CNT_수리서비스,SC_CNT_숙박,SC_CNT_신변잡화,SC_CNT_연료판매,SC_CNT_요식업소,SC_CNT_유통업,SC_CNT_음료식품,SC_CNT_의료기관,SC_CNT_의복,SC_CNT_자동차정비,SC_CNT_자동차판매,SC_CNT_전기,SC_CNT_주방용구,SC_CNT_직물,SC_AMT_가구,SC_AMT_가전,SC_AMT_광학제품,SC_AMT_레저업소,SC_AMT_레저용품,SC_AMT_문화취미,SC_AMT_보건위생,SC_AMT_사무통신,SC_AMT_서적문구,SC_AMT_수리서비스,SC_AMT_숙박,SC_AMT_신변잡화,SC_AMT_연료판매,SC_AMT_요식업소,SC_AMT_유통업,SC_AMT_음료식품,SC_AMT_의료기관,SC_AMT_의복,SC_AMT_자동차정비,SC_AMT_자동차판매,SC_AMT_전기,SC_AMT_주방용구,SC_AMT_직물,GS_매출지수,GS_식사,GS_간식,GS_마실거리,GS_홈/리빙,GS_헬스/뷰티,GS_취미/여가활동,GS_사회활동,GS_임신육아,TMST_MORNING,TMST_NOON,TMST_AFTNOON,TMST_NIGHT,TMST_DAWN,HDONG_GU
0,2019-02-01,범어1동,6.0,64.0,15.0,20.0,3.0,14.0,0.0,29.0,12.0,19.0,0.0,31.0,0.0,166.0,0.0,599.0,237.0,0.0,0.0,26.0,46.0,0.0,908.0,3938.0,3056.0,627.0,1521.0,0.0,197.0,0.0,0.0,0.0,0.0,0.0,12158.0,0.0,25635.0,0.0,43048.0,12552.0,0.0,0.0,942.0,2010.0,0.0,29133.0,117524.0,44322.0,16206.0,132778.0,0.0,2443.0,0.0,0.0,0.0,0.0,1.08,34.020,18.144,42.228,1.728,10.044,0.108,1.296,0.324,6613.0625,6470.653333,6570.0875,5895.6525,1109.99625,대구 수성구
1,2019-02-01,동인동,6.0,41.0,6.0,20.0,0.0,15.0,1.0,21.0,9.0,21.0,0.0,0.0,5.0,97.0,5.0,0.0,73.0,20.0,0.0,5.0,54.0,0.0,330.0,2640.0,2398.0,379.0,1022.0,0.0,52.0,0.0,5.0,15.0,0.0,0.0,0.0,27.0,3268.0,454.0,0.0,8949.0,804.0,0.0,26.0,2001.0,0.0,13469.0,63156.0,20924.0,30160.0,136270.0,0.0,2429.0,0.0,1320.0,275.0,0.0,0.77,28.875,12.936,25.256,4.774,4.928,0.154,0.154,0.000,6020.9925,5671.933333,6034.2100,4751.1775,927.58250,대구 중구
2,2019-02-01,범어2동,12.0,53.0,14.0,23.0,0.0,20.0,0.0,36.0,19.0,28.0,0.0,0.0,29.0,400.0,30.0,537.0,558.0,72007.0,379.0,54.0,960.0,48.0,566.0,184667.0,61985.0,1969.0,3316.0,142.0,6409.0,0.0,0.0,5.0,0.0,0.0,0.0,966.0,20710.0,8766.0,7935.0,31642.0,514292.0,7774.0,1495.0,115004.0,51541.0,46888.0,1781639.0,2300513.0,69906.0,302176.0,14825.0,111690.0,0.0,0.0,276.0,0.0,1.06,45.368,20.776,28.620,2.650,7.208,0.000,1.060,0.424,6584.1325,6379.133333,7065.8275,5671.3975,1115.14500,대구 수성구
3,2019-02-01,소공동,3.0,48.0,35.0,15.0,0.0,19.0,0.0,25.0,10.0,26.0,0.0,0.0,29.0,400.0,30.0,537.0,558.0,72007.0,379.0,54.0,960.0,48.0,566.0,184667.0,61985.0,1969.0,3316.0,142.0,6409.0,0.0,0.0,5.0,0.0,0.0,0.0,966.0,20710.0,8766.0,7935.0,31642.0,514292.0,7774.0,1495.0,115004.0,51541.0,46888.0,1781639.0,2300513.0,69906.0,302176.0,14825.0,111690.0,0.0,0.0,276.0,0.0,1.67,48.096,31.062,64.128,7.014,13.527,0.668,1.837,0.835,12642.9050,12738.570000,12449.9950,8822.2900,3058.72000,서울 중구
4,2019-02-01,범어3동,4.0,46.0,10.0,21.0,1.0,31.0,1.0,39.0,11.0,22.0,0.0,0.0,0.0,204.0,15.0,47.0,36.0,5.0,47.0,96.0,20.0,0.0,0.0,2597.0,2336.0,585.0,1466.0,0.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9299.0,276.0,1173.0,594.0,5.0,1004.0,993.0,975.0,0.0,0.0,87056.0,32592.0,13834.0,100590.0,0.0,877.0,0.0,0.0,0.0,0.0,0.96,29.664,20.736,31.392,1.632,8.640,0.672,0.288,2.976,3900.8175,3779.263333,4149.5600,3604.3350,686.79000,대구 수성구


In [ ]:
#all.to_csv('./gdrive/My Drive/빅콘 대상팀/data/all_amt.csv', sep=',',index=False)